In [122]:
import csv
import sys
import re
import pandas as pd
import requests
import numpy as np
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.iolib.summary2 import summary_col


In [84]:
cd /Users/hazelchui/Desktop/UChicago/Perspectives

/Users/hazelchui/Desktop/UChicago/Perspectives


In [17]:
oscars_lst = pd.read_csv("/Users/hazelchui/Desktop/UChicago/Perspectives/final_oscars_list.csv")
oscars_lst = oscars_lst.fillna(0)
oscars_lst['release_year'] = (oscars_lst['year']-1).astype(str)
oscars_lst['movie'] = oscars_lst['movie'].str.extract(r'(.*?)\xa0–')
oscars_lst.rename(columns={'movie':'movie_name'}, inplace = True)
oscars_lst = oscars_lst.dropna(subset=['movie_name'])
oscars_lst


,year,movie_name,best_picture,best_picture_won,best_animated_feature,best_animated_feature_won,best_foreign,best_foreign_won,best_doc,best_doc_won,best_doc_short,best_doc_short_won,best_live_short,best_live_short_won,best_animated_short,best_animated_short_won,release_year
0,2015,Birdman or (The Unexpected Virtue of Ignorance),1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2014
1,2015,American Sniper,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2014
2,2015,Boyhood,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2014
3,2015,The Grand Budapest Hotel,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2014
4,2015,The Imitation Game,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2014
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
484,2008,Peter & the Wolf,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,2007
485,2008,Even Pigeons Go to Heaven (Même les pigeons vo...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2007
486,2008,I Met the Walrus,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2007
487,2008,Madame Tutli-Putli,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2007


### Cleaning reviews

In [3]:
# Clean for review.csv (185789 rows)
review = pd.read_csv("/Users/hazelchui/Desktop/UChicago/Perspectives/reviews.csv")
review

,release_year,movie_id,movie_name,date,user_name,review,rating
0,2014,tt2562232,Birdman or (The Unexpected Virtue of Ignorance),19 September 2018,PIST-OFF,With all due respect to Eastwood's American Sn...,10
1,2014,tt2562232,Birdman or (The Unexpected Virtue of Ignorance),20 July 2015,SnoopyStyle,Riggan Thomson (Michael Keaton) is famous for ...,8
2,2014,tt2562232,Birdman or (The Unexpected Virtue of Ignorance),1 January 2015,CalRhys,"Whilst viewing 'Birdman', I spent the first ho...",NaN
3,2014,tt2562232,Birdman or (The Unexpected Virtue of Ignorance),9 June 2017,TheLittleSongbird,Despite the near-universal acclaim from critic...,9
4,2014,tt2562232,Birdman or (The Unexpected Virtue of Ignorance),24 February 2015,apbryant,I have to say I am shocked and how many bad re...,10
...,...,...,...,...,...,...,...
185784,2007,tt0874952,My Love (Moya Lyubov),6 September 2014,Vincentiu,impressionist art. delicate theme. Alexandr Pe...,NaN
185785,2007,tt0874952,My Love (Moya Lyubov),18 May 2017,Kirpianuscus,it is a great travel. across a feeling. across...,NaN
185786,2007,tt0874952,My Love (Moya Lyubov),17 July 2015,Horst_In_Translation,This is a Russian animated short film that run...,4
185787,2007,tt0874952,My Love (Moya Lyubov),21 June 2018,Jithindurden,The love and lust of an adolescent boy. The in...,9


In [12]:
# Drop reviews without rating 
review = review.dropna(subset=['rating'])

# Drop duplicated reviews (170590 rows)
review = review.drop_duplicates(subset=['review'], keep='first')

# Drop heading (170589 rows)
review = review.loc[(review['date'] != 'date')]

# Unique movies (452)
len(review.movie_name.unique().tolist())

452

In [13]:
# date format
review['date'] = pd.to_datetime(review['date'], format='%d %B %Y')

# Sort by movie_id and date
review = review.sort_values(by=['movie_id', 'date'])

# 3 months before Oscars
review.loc[(review['date'] >= "2007-11-24") & ("2008-02-24" > review['date'])& (review["release_year"] == '2007'),'period'] = 0
review.loc[(review['date'] >= "2008-11-22") & ("2009-02-22" > review['date'])& (review["release_year"] == '2008'),'period'] = 0
review.loc[(review['date'] >= "2009-12-07") & ("2010-03-07" > review['date'])& (review["release_year"] == '2009'),'period'] = 0
review.loc[(review['date'] >= "2010-11-27") & ("2011-02-27" > review['date'])& (review["release_year"] == '2010'),'period'] = 0
review.loc[(review['date'] >= "2011-11-26") & ("2012-02-26" > review['date'])& (review["release_year"] == '2011'),'period'] = 0
review.loc[(review['date'] >= "2012-11-24") & ("2013-02-24" > review['date'])& (review["release_year"] == '2012'),'period'] = 0
review.loc[(review['date'] >= "2013-12-02") & ("2014-03-02" > review['date'])& (review["release_year"] == '2013'),'period'] = 0
review.loc[(review['date'] >= "2014-11-22") & ("2015-02-22" > review['date'])& (review["release_year"] == '2014'),'period'] = 0
review.loc[(review['date'] >= "2015-11-28") & ("2016-02-28" > review['date'])& (review["release_year"] == '2015'),'period'] = 0
review.loc[(review['date'] >= "2016-11-26") & ("2017-02-26" > review['date'])& (review["release_year"] == '2016'),'period'] = 0
review.loc[(review['date'] >= "2017-12-04") & ("2018-03-04" > review['date'])& (review["release_year"] == '2017'),'period'] = 0
review.loc[(review['date'] >= "2018-11-24") & ("2019-02-24" > review['date'])& (review["release_year"] == '2018'),'period'] = 0
review.loc[(review['date'] >= "2019-11-09") & ("2020-02-09" > review['date'])& (review["release_year"] == '2019'),'period'] = 0

# 3 months after Oscars
review.loc[(review['date'] >= "2008-02-24") & ("2008-05-24" > review['date'])& (review["release_year"] == '2007'),'period'] = 1
review.loc[(review['date'] >= "2009-02-22") & ("2009-05-22" > review['date'])& (review["release_year"] == '2008'),'period'] = 1
review.loc[(review['date'] >= "2010-03-07") & ("2010-06-07" > review['date'])& (review["release_year"] == '2009'),'period'] = 1
review.loc[(review['date'] >= "2011-02-27") & ("2011-05-27" > review['date'])& (review["release_year"] == '2010'),'period'] = 1
review.loc[(review['date'] >= "2012-02-26") & ("2012-05-26" > review['date'])& (review["release_year"] == '2011'),'period'] = 1
review.loc[(review['date'] >= "2013-02-24") & ("2013-05-24" > review['date'])& (review["release_year"] == '2012'),'period'] = 1
review.loc[(review['date'] >= "2014-03-02") & ("2014-06-02" > review['date'])& (review["release_year"] == '2013'),'period'] = 1
review.loc[(review['date'] >= "2015-02-22") & ("2015-05-22" > review['date'])& (review["release_year"] == '2014'),'period'] = 1
review.loc[(review['date'] >= "2016-02-28") & ("2016-05-28" > review['date'])& (review["release_year"] == '2015'),'period'] = 1
review.loc[(review['date'] >= "2017-02-26") & ("2017-05-26" > review['date'])& (review["release_year"] == '2016'),'period'] = 1
review.loc[(review['date'] >= "2018-03-04") & ("2018-06-04" > review['date'])& (review["release_year"] == '2017'),'period'] = 1
review.loc[(review['date'] >= "2019-02-24") & ("2019-05-24" > review['date'])& (review["release_year"] == '2018'),'period'] = 1
review.loc[(review['date'] >= "2020-02-09") & ("2020-05-09" > review['date'])& (review["release_year"] == '2019'),'period'] = 1


In [14]:
# Only need reviews within 3-month range (64508 rows)
final = review.loc[(review['period'] == 0.0) | (review['period'] == 1.0)]
final = final.astype({"rating": int, "period": int})

# 437 unique movies (some movies only have reviews out of 3-month range)
len(final.movie_name.unique().tolist())

437

In [15]:
# Cal rating mean (groupby ['movie_name', 'period'])
s = final.groupby(['movie_name', 'period','movie_id'])['rating'].mean()
rating_df = s.to_frame()
rating_df = rating_df.reset_index()

# 805 rows, meaning that some movies have no reviws in either 0/1 periodsd
rating_df

,movie_name,period,movie_id,rating
0,(Il Supplente) The Substitute,0,tt0922627,8.500000
1,(Il Supplente) The Substitute,1,tt0922627,10.000000
2,12 Years a Slave,0,tt2024544,7.750000
3,12 Years a Slave,1,tt2024544,7.086614
4,127 Hours,0,tt1542344,7.726190
...,...,...,...,...
800,Wreck-It Ralph,1,tt1772341,7.844444
801,Zero Dark Thirty,0,tt1790885,6.379205
802,Zero Dark Thirty,1,tt1790885,5.942857
803,Zootopia,0,tt2948356,7.939394


In [16]:
# Count rating num (groupby ['movie_name', 'period'])
c = final.groupby(['movie_name', 'period'])['rating'].size()
count_df = c.to_frame()
count_df.rename(columns={'rating':'review_num'}, inplace = True)
count_df = count_df.reset_index()
count_df

,movie_name,period,review_num
0,(Il Supplente) The Substitute,0,2
1,(Il Supplente) The Substitute,1,1
2,12 Years a Slave,0,240
3,12 Years a Slave,1,127
4,127 Hours,0,168
...,...,...,...
800,Wreck-It Ralph,1,45
801,Zero Dark Thirty,0,327
802,Zero Dark Thirty,1,105
803,Zootopia,0,33


In [21]:
# 818 rows, somes movies miss either period 1/0
merge_1 = oscars_lst.merge(rating_df, how = 'left', on= 'movie_name')
merge_1


,year,movie_name,best_picture,best_picture_won,best_animated_feature,best_animated_feature_won,best_foreign,best_foreign_won,best_doc,best_doc_won,best_doc_short,best_doc_short_won,best_live_short,best_live_short_won,best_animated_short,best_animated_short_won,release_year,period,movie_id,rating
0,2015,Birdman or (The Unexpected Virtue of Ignorance),1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2014,0.0,tt2562232,6.439394
1,2015,Birdman or (The Unexpected Virtue of Ignorance),1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2014,1.0,tt2562232,5.264331
2,2015,American Sniper,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2014,0.0,tt2179136,6.620818
3,2015,American Sniper,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2014,1.0,tt2179136,5.882353
4,2015,Boyhood,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2014,0.0,tt1065073,5.950178
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
813,2008,Even Pigeons Go to Heaven (Même les pigeons vo...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2007,0.0,tt1066329,8.666667
814,2008,I Met the Walrus,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2007,0.0,tt1156511,6.000000
815,2008,Madame Tutli-Putli,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2007,0.0,tt1029440,7.800000
816,2008,My Love (Moya Lyubov),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2007,0.0,tt0874952,8.333333


In [25]:
# drop na in rating (some movies have no rating at all)
merge_1 = merge_1.dropna(subset=['rating'])
# # drop movies without pair (688 rows)
merge_1 = merge_1[merge_1.duplicated('movie_name') | merge_1.duplicated('movie_name', keep='last')]

In [29]:
# change dtype & reset index
merge_1 = merge_1.astype({"period": int})
merge_1.reset_index(drop=True, inplace=True)
# 343 unique movies
len(merge_1['movie_name'].unique().tolist())

343

In [30]:
# merge with review num (should have no isna)
merge_2 = merge_1.merge(count_df, how = 'left', on= ['movie_name', 'period'])
merge_2

,year,movie_name,best_picture,best_picture_won,best_animated_feature,best_animated_feature_won,best_foreign,best_foreign_won,best_doc,best_doc_won,...,best_doc_short_won,best_live_short,best_live_short_won,best_animated_short,best_animated_short_won,release_year,period,movie_id,rating,review_num
0,2015,Birdman or (The Unexpected Virtue of Ignorance),1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2014,0,tt2562232,6.439394,462
1,2015,Birdman or (The Unexpected Virtue of Ignorance),1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2014,1,tt2562232,5.264331,314
2,2015,American Sniper,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2014,0,tt2179136,6.620818,538
3,2015,American Sniper,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2014,1,tt2179136,5.882353,136
4,2015,Boyhood,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2014,0,tt1065073,5.950178,281
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
683,2008,The Tonto Woman,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,2007,1,tt1176466,6.500000,2
684,2008,Peter & the Wolf,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,2007,0,tt0863136,6.333333,3
685,2008,Peter & the Wolf,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,2007,1,tt0863136,9.666667,3
686,2008,My Love (Moya Lyubov),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,2007,0,tt0874952,8.333333,3


In [34]:
# Create a winner dummy 
merge_2.loc[(merge_2['best_picture_won'] == 1.0),'winner'] = 1
merge_2.loc[(merge_2['best_animated_feature_won'] == 1.0),'winner'] = 1
merge_2.loc[(merge_2['best_foreign_won'] == 1.0),'winner'] = 1
merge_2.loc[(merge_2['best_doc_won'] == 1.0),'winner'] = 1
merge_2.loc[(merge_2['best_doc_short_won'] == 1.0),'winner'] = 1
merge_2.loc[(merge_2['best_live_short_won'] == 1.0),'winner'] = 1
merge_2.loc[(merge_2['best_animated_short_won'] == 1.0),'winner'] = 1
merge_2 = merge_2.fillna(0)
# 688 rows (3)
merge_2 = merge_2.astype({"winner": int})
merge_2

,year,movie_name,best_picture,best_picture_won,best_animated_feature,best_animated_feature_won,best_foreign,best_foreign_won,best_doc,best_doc_won,...,best_live_short,best_live_short_won,best_animated_short,best_animated_short_won,release_year,period,movie_id,rating,review_num,winner
0,2015,Birdman or (The Unexpected Virtue of Ignorance),1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2014,0,tt2562232,6.439394,462,1
1,2015,Birdman or (The Unexpected Virtue of Ignorance),1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2014,1,tt2562232,5.264331,314,1
2,2015,American Sniper,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2014,0,tt2179136,6.620818,538,0
3,2015,American Sniper,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2014,1,tt2179136,5.882353,136,0
4,2015,Boyhood,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2014,0,tt1065073,5.950178,281,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
683,2008,The Tonto Woman,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,2007,1,tt1176466,6.500000,2,0
684,2008,Peter & the Wolf,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,2007,0,tt0863136,6.333333,3,1
685,2008,Peter & the Wolf,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,2007,1,tt0863136,9.666667,3,1
686,2008,My Love (Moya Lyubov),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,2007,0,tt0874952,8.333333,3,0


In [35]:
# Remove movies with less than 5 reviews 
merge_2 = merge_2.loc[merge_2['review_num']>=5]
# Remove movies without pair
merge_2 = merge_2[merge_2.duplicated('movie_name') | merge_2.duplicated('movie_name', keep='last')]
# 426 rows, 213 movies 
# Save as csv
#merge_2.to_csv('apr25_review_more_5.csv', index = False)


,year,movie_name,best_picture,best_picture_won,best_animated_feature,best_animated_feature_won,best_foreign,best_foreign_won,best_doc,best_doc_won,...,best_live_short,best_live_short_won,best_animated_short,best_animated_short_won,release_year,period,movie_id,rating,review_num,winner
0,2015,Birdman or (The Unexpected Virtue of Ignorance),1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2014,0,tt2562232,6.439394,462,1
1,2015,Birdman or (The Unexpected Virtue of Ignorance),1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2014,1,tt2562232,5.264331,314,1
2,2015,American Sniper,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2014,0,tt2179136,6.620818,538,0
3,2015,American Sniper,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2014,1,tt2179136,5.882353,136,0
4,2015,Boyhood,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2014,0,tt1065073,5.950178,281,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
673,2008,Taxi to the Dark Side,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,2007,1,tt0854678,8.000000,5,1
674,2008,No End in Sight,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,2007,0,tt0912593,9.111111,9,0
675,2008,No End in Sight,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,2007,1,tt0912593,8.600000,5,0
676,2008,Sicko,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,2007,0,tt0386032,7.885714,35,0


In [74]:
df = pd.read_csv('apr25_review_more_5.csv')

# Choose control movies with similar prior rating as the winner in the group
df = df.loc[df['period'] == 0]
df.loc[(df['winner']==1), 'winner_rating'] = df['rating']
df['winner_rating'] = df.groupby(["year", 'best_picture', 'best_animated_feature', 'best_foreign', 'best_doc', 'best_doc_short','best_live_short', 'best_animated_short'])['winner_rating'].ffill()

df

,year,movie_name,best_picture,best_picture_won,best_animated_feature,best_animated_feature_won,best_foreign,best_foreign_won,best_doc,best_doc_won,...,best_live_short_won,best_animated_short,best_animated_short_won,release_year,period,movie_id,rating,review_num,winner,winner_rating
0,2015,Birdman or (The Unexpected Virtue of Ignorance),1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,2014,0,tt2562232,6.439394,462,1,6.439394
2,2015,American Sniper,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,2014,0,tt2179136,6.620818,538,0,6.439394
4,2015,Boyhood,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,2014,0,tt1065073,5.950178,281,0,6.439394
6,2015,The Grand Budapest Hotel,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,2014,0,tt2278388,7.177083,96,0,6.439394
8,2015,The Imitation Game,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,2014,0,tt2084970,7.763158,304,0,6.439394
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
416,2008,Katyń (Poland) in Polish,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,2007,0,tt0879843,7.900000,10,0,6.875000
418,2008,Mongol (Kazakhstan) in Russian,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,2007,0,tt0416044,4.300000,10,0,6.875000
420,2008,Taxi to the Dark Side,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,2007,0,tt0854678,8.800000,5,1,8.800000
422,2008,No End in Sight,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,2007,0,tt0912593,9.111111,9,0,8.800000


In [75]:
# Some 'winner_rating' with NaN since winner is not in the group
# Winner was deleted since they have less than 5 reviews

# drop movies without winner in the group
df = df.dropna()

# 200 movies 

In [76]:
# Choose movies with similar prior rating
df['diff'] = abs(df['winner_rating'] - df['rating'])

# drop winner: 152 rows
df = df.loc[df['diff']!=0]
df.head(20)

/var/folders/nk/mc55cdxs3h39bqv94923ds1m0000gn/T/ipykernel_1474/3874730948.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['diff'] = abs(df['winner_rating'] - df['rating'])


,year,movie_name,best_picture,best_picture_won,best_animated_feature,best_animated_feature_won,best_foreign,best_foreign_won,best_doc,best_doc_won,...,best_animated_short,best_animated_short_won,release_year,period,movie_id,rating,review_num,winner,winner_rating,diff
2,2015,American Sniper,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,2014,0,tt2179136,6.620818,538,0,6.439394,0.181424
4,2015,Boyhood,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,2014,0,tt1065073,5.950178,281,0,6.439394,0.489216
6,2015,The Grand Budapest Hotel,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,2014,0,tt2278388,7.177083,96,0,6.439394,0.737689
8,2015,The Imitation Game,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,2014,0,tt2084970,7.763158,304,0,6.439394,1.323764
10,2015,Selma,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,2014,0,tt1020072,7.900000,110,0,6.439394,1.460606
12,2015,The Theory of Everything,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,2014,0,tt2980516,7.828431,204,0,6.439394,1.389037
14,2015,Whiplash,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,2014,0,tt2582802,8.540000,250,0,6.439394,2.100606
18,2015,The Boxtrolls,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,2014,0,tt0787474,6.969697,33,0,7.951724,0.982027
20,2015,How to Train Your Dragon 2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,2014,0,tt1646971,7.878788,33,0,7.951724,0.072936
22,2015,Song of the Sea,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,2014,0,tt1865505,8.666667,6,0,7.951724,0.714943


In [77]:
# Find movies with the smallest diff with the winner (38 movies)
min_df = df.groupby(['year', 'best_picture', 'best_animated_feature', 'best_foreign', 'best_doc', 'best_doc_short','best_live_short', 'best_animated_short'])['diff'].min()
min_df = min_df.to_frame()
min_df['min'] = 1
min_df = min_df.reset_index()
min_df

,year,best_picture,best_animated_feature,best_foreign,best_doc,best_doc_short,best_live_short,best_animated_short,diff,min
0,2008,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.311111,1
1,2008,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.025000,1
2,2008,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.765714,1
3,2008,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.023511,1
4,2009,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.053549,1
5,2009,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.416071,1
6,2010,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.922705,1
7,2010,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.966667,1
8,2010,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.332866,1
9,2011,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.128788,1


In [78]:
# Merge with df
df = pd.merge(df, min_df[['year','diff', 'best_picture', 'best_animated_feature', 'best_foreign', 'best_doc', 'best_doc_short','best_live_short', 'best_animated_short', 'min']], how = 'left', on= ['year','diff', 'best_picture', 'best_animated_feature', 'best_foreign', 'best_doc', 'best_doc_short','best_live_short', 'best_animated_short'])


In [79]:
df.head(20)

,year,movie_name,best_picture,best_picture_won,best_animated_feature,best_animated_feature_won,best_foreign,best_foreign_won,best_doc,best_doc_won,...,best_animated_short_won,release_year,period,movie_id,rating,review_num,winner,winner_rating,diff,min
0,2015,American Sniper,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,2014,0,tt2179136,6.620818,538,0,6.439394,0.181424,1.0
1,2015,Boyhood,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,2014,0,tt1065073,5.950178,281,0,6.439394,0.489216,NaN
2,2015,The Grand Budapest Hotel,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,2014,0,tt2278388,7.177083,96,0,6.439394,0.737689,NaN
3,2015,The Imitation Game,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,2014,0,tt2084970,7.763158,304,0,6.439394,1.323764,NaN
4,2015,Selma,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,2014,0,tt1020072,7.900000,110,0,6.439394,1.460606,NaN
5,2015,The Theory of Everything,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,2014,0,tt2980516,7.828431,204,0,6.439394,1.389037,NaN
6,2015,Whiplash,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,2014,0,tt2582802,8.540000,250,0,6.439394,2.100606,NaN
7,2015,The Boxtrolls,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,2014,0,tt0787474,6.969697,33,0,7.951724,0.982027,NaN
8,2015,How to Train Your Dragon 2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,2014,0,tt1646971,7.878788,33,0,7.951724,0.072936,1.0
9,2015,Song of the Sea,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,2014,0,tt1865505,8.666667,6,0,7.951724,0.714943,NaN


In [86]:
# Merge with main review df
full = pd.read_csv('apr25_review_more_5.csv')
full = pd.merge(full, df[['movie_name', 'min']], how = 'left', on= ['movie_name'])
full

,year,movie_name,best_picture,best_picture_won,best_animated_feature,best_animated_feature_won,best_foreign,best_foreign_won,best_doc,best_doc_won,...,best_live_short_won,best_animated_short,best_animated_short_won,release_year,period,movie_id,rating,review_num,winner,min
0,2015,Birdman or (The Unexpected Virtue of Ignorance),1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,2014,0,tt2562232,6.439394,462,1,NaN
1,2015,Birdman or (The Unexpected Virtue of Ignorance),1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,2014,1,tt2562232,5.264331,314,1,NaN
2,2015,American Sniper,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,2014,0,tt2179136,6.620818,538,0,1.0
3,2015,American Sniper,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,2014,1,tt2179136,5.882353,136,0,1.0
4,2015,Boyhood,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,2014,0,tt1065073,5.950178,281,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
421,2008,Taxi to the Dark Side,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,2007,1,tt0854678,8.000000,5,1,NaN
422,2008,No End in Sight,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,2007,0,tt0912593,9.111111,9,0,1.0
423,2008,No End in Sight,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,2007,1,tt0912593,8.600000,5,0,1.0
424,2008,Sicko,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,2007,0,tt0386032,7.885714,35,0,NaN


In [104]:
# Keep comparable movies (174 rows)
final_2 = full.loc[(full['winner']==1) |(full['min']==1.0)]
#final_2.to_csv('final_data_no_match.csv', index = False)

### With 1:1 match

In [92]:
match = final_2.groupby(['year', 'best_picture', 'best_animated_feature', 'best_foreign', 'best_doc', 'best_doc_short','best_live_short', 'best_animated_short']).size()
match = match.to_frame()
match = match.reset_index()
match


,year,best_picture,best_animated_feature,best_foreign,best_doc,best_doc_short,best_live_short,best_animated_short,0
0,2008,0.0,0.0,0.0,1.0,0.0,0.0,0.0,4
1,2008,0.0,0.0,1.0,0.0,0.0,0.0,0.0,4
2,2008,0.0,1.0,0.0,0.0,0.0,0.0,0.0,4
3,2008,1.0,0.0,0.0,0.0,0.0,0.0,0.0,4
4,2009,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2
5,2009,0.0,1.0,0.0,0.0,0.0,0.0,0.0,4
6,2009,1.0,0.0,0.0,0.0,0.0,0.0,0.0,4
7,2010,0.0,0.0,0.0,1.0,0.0,0.0,0.0,4
8,2010,0.0,0.0,1.0,0.0,0.0,0.0,0.0,4
9,2010,1.0,0.0,0.0,0.0,0.0,0.0,0.0,4


In [93]:
final_data = pd.merge(final_2, match[['year', 'best_picture', 'best_animated_feature', 'best_foreign', 'best_doc', 'best_doc_short','best_live_short', 'best_animated_short',0]], how = 'left', on= ['year', 'best_picture', 'best_animated_feature', 'best_foreign', 'best_doc', 'best_doc_short','best_live_short', 'best_animated_short'])


In [102]:
# 156 rows
final_data = final_data.loc[final_data[0]==4]
#final_data.to_csv('final_data.csv', index = False)

# 10-year data: 2010 to 2019 data (116 rows)
final_data_2 = final_data.loc[(final_data['year']!= 2020) & (final_data['year']!= 2008) & (final_data['year']!= 2009)]
#final_data_2.to_csv('final_data_2010_2019.csv', index=False)

In [114]:
final_data_2

,year,movie_name,best_picture,best_picture_won,best_animated_feature,best_animated_feature_won,best_foreign,best_foreign_won,best_doc,best_doc_won,...,best_animated_short,best_animated_short_won,release_year,period,movie_id,rating,review_num,winner,min,0
0,2015,Birdman or (The Unexpected Virtue of Ignorance),1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,2014,0,tt2562232,6.439394,462,1,NaN,4
1,2015,Birdman or (The Unexpected Virtue of Ignorance),1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,2014,1,tt2562232,5.264331,314,1,NaN,4
2,2015,American Sniper,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,2014,0,tt2179136,6.620818,538,0,1.0,4
3,2015,American Sniper,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,2014,1,tt2179136,5.882353,136,0,1.0,4
4,2015,Big Hero 6,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,2014,0,tt2245084,7.951724,145,1,NaN,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143,2010,"A Prophet (France) in French, Corsican and Arabic",0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,2009,1,tt1235166,7.750000,24,0,1.0,4
144,2010,The Cove,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,2009,0,tt1313104,8.478261,23,1,NaN,4
145,2010,The Cove,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,2009,1,tt1313104,6.600000,5,1,NaN,4
146,2010,"Food, Inc.",0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,2009,0,tt1286537,7.555556,9,0,1.0,4
